In [48]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [212]:
# 97%
data = pd.read_csv(
    '../data/feature_sets/rubberbanddata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/rubberbanddoc.csv'
)

# 97%
data = pd.read_csv(
    '../data/feature_sets/celciusdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/celciusdoc.csv'
)


# 86%
data = pd.read_csv(
    '../data/feature_sets/saltwaterdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/saltwaterdoc.csv'
)

# 79%
data = pd.read_csv(
    '../data/feature_sets/pitchesdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/pitchesdoc.csv'
)

# 100%
data = pd.read_csv(
    '../data/feature_sets/sugardata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/sugardoc.csv'
)
# 
# 71%
data = pd.read_csv(
    '../data/feature_sets/standarddata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/standarddoc.csv'
)

# 96%
data = pd.read_csv(
    '../data/feature_sets/celciusdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/celciusdoc.csv'
)

# 70%
data = pd.read_csv(
    '../data/feature_sets/loose_wiresdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/loose_wiresdoc.csv'
)

# 43%%
data = pd.read_csv(
    '../data/feature_sets/batterydata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/batterydoc.csv'
)

# 74%%
data = pd.read_csv(
    '../data/feature_sets/cleardata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/cleardoc.csv'
)

# 79%
data = pd.read_csv(
    '../data/feature_sets/circuitdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/circuitdoc.csv'
)

# 57%
data = pd.read_csv(
    '../data/feature_sets/heat_sinkdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/heat_sinkdoc.csv'
)

# 57%
data = pd.read_csv(
    '../data/feature_sets/dont_knowdata.csv'
)
doc = pd.read_csv(
    '../data/feature_sets/dont_knowdoc.csv'
)

In [213]:
X_train, X_test, y_train, y_test = train_test_split(data, doc.label, test_size = .8, stratify = doc.label)

In [214]:
clf = LogisticRegression()
# clf = GaussianNB()
# clf = GradientBoostingClassifier()

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8

In [215]:
confusion_matrix(y_test, y_pred)

array([[24,  0],
       [ 6,  0]], dtype=int64)

In [180]:
doc.label.value_counts()

0    26
1    10
Name: label, dtype: int64

In [128]:
X_train

,wordcount,sentence_count,stem_g_similarity,stem_j_similarity,stem_c_similarity,stem_ordered_g_similarity,stem_ordered_j_similarity,stem_ordered_c_similarity,q_stemmed_has_bill,q_stemmed_has_degre,q_stemmed_has_celsiu,q_stemmed_has_28,q_stemmed_has_warm,q_stemmed_has_temperatur,q_stem_ordered_has_28,q_stem_ordered_has_bill,q_stem_ordered_has_celsiu,q_stem_ordered_has_degre,q_stem_ordered_has_temperatur,q_stem_ordered_has_warm
24,0.4,0.0,0.375000,0.125000,0.2357,0.375000,0.125000,0.2357,0,0,1,0,0,0,0,0,1,0,0,0
17,0.0,0.0,0.166667,0.166667,0.4082,0.166667,0.166667,0.4082,0,0,1,0,0,0,0,0,1,0,0,0
27,0.0,0.0,0.166667,0.166667,0.4082,0.166667,0.166667,0.4082,0,0,1,0,0,0,0,0,1,0,0,0
32,0.0,0.0,0.142857,0.000000,0.0000,0.142857,0.000000,0.0000,0,0,0,0,0,0,0,0,0,0,0,0
21,0.6,0.0,0.571429,0.428571,0.6124,0.571429,0.428571,0.6124,0,1,0,1,1,0,1,0,0,1,0,1
8,0.2,0.0,0.285714,0.142857,0.2887,0.285714,0.142857,0.2887,0,0,0,1,0,0,1,0,0,0,0,0
6,0.2,0.0,0.333333,0.333333,0.5774,0.333333,0.333333,0.5774,0,1,1,0,0,0,0,0,1,1,0,0


In [129]:
y_train

24    1
17    1
27    1
32    0
21    0
8     0
6     1
Name: label, dtype: int64